<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; text-align: left; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br />Jan Attig</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; font-size: 30px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; font-size: 30px; padding:0px;">Übungsblatt 9</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; font-size: 20px; padding:0px; margin-bottom: 20px;">Sommersemester 2020</h3>
<hr>

**Website** [http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml)

**Abgabe**:  22.6.2020 bis 13:00 Uhr

**Besprechung**:  24.6.2020 - 26.6.2020

**Name**: <font color="red">Bitte geben Sie ihren Namen an</font>

**Matrikelnummer**: <font color="red">Bitte geben Sie ihre Matrikelnummer an</font>

<table style="border-style: none; width: 100%"><tr style="border-style: none;">
<td style="border-style: none; width: 1%; text-align: left; font-size: 25px; font-weight: bold; text-decoration: underline">Aufgaben auf Übungsblatt 9</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 15px"></td></tr></table>

- Hofstadter Butterfly [8 Punkte]
- Der Geist in der Matrix [12 Punkte]
- **(Bonus) Wie eine Düne im Wind** [10 Bonus-Punkte]


<table style="border-style: none; width: 100%"><tr style="border-style: none;">
<td style="border-style: none; width: 1%; text-align: left; font-size: 25px; font-weight: bold; text-decoration: underline">Wie eine Düne im Wind</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 15px">[10 Bonus-Punkte]</td></tr></table>

Beim Begriff **Düne** denkt man sicher vor allem an trockene Sandwüsten und heiße Sonne die vom Himmel herab brennt, allerdings kommen Dünen auch teilweise in unseren Breiten in Form von sogenannten [Binnendünen](https://de.wikipedia.org/wiki/Binnend%C3%BCne) vor. Die von Köln aus nächste solche Binnendüne findet sich in [Bonn Tannenbusch](https://www.biostation-bonn-rheinerft.de/startseite/projekte/projekte-bonn/duene-tannenbusch/naturschutzgebiet-duene-tannenbusch). Als Relikt aus der letzten Eiszeit bietet diese Düne nicht nur eine einzigartige Formation im Rheintal, sondern beherbergt auch auf Grund des sandigen Bodens eine besondere Artenvielfalt. Allerdings gibt es auch ernste Probleme: Durch Wind fängt die Düne an zu wandern, eine Bewegung die sich praktisch nicht aufhalten lässt.

In dieser Bonus-Aufgabe wollen wir uns die **Formation und Wanderbewegung** von Dünen anschauen und modellieren. Dazu betrachten wir das dynamische System, das sich aus Sandkörnern und Wind ergibt und bringen es als Simulation auf den Computer. Es sei dabei angemerkt, dass die Computersimulation von Dünen ein eigenes Fachgebiet der Physik ist und teilweise seit Jahrzenten untersucht wird, wie z.b. in [diesem](https://pubs.geoscienceworld.org/gsa/geology/article/23/12/1107/206230/eolian-dunes-computer-simulations-and-attractor) Artikel beschrieben. Im Folgenden möchten wir Ihnen eine vereinfachte Version einer Simulationstechnik näher bringen.

### Modell Grundlagen

Um das Sand-Wind System zu simulieren, verwenden wir eine Abstraktion die einem zellulären Automaten ähnelt. Wir diskretisieren den Raum in ein Quadratgitter und den Sand in kleine Häufchen. Der Sand türmt sich auf dem Feld $i,j$ zu einer Höhe $h_{ij} \in \mathbb{N}$ auf, die jeweils die Anzahl der darauf vorkommenden Häufchen beschreibt (siehe Skizze weiter unten). In Julia lässt sich damit eine Sandkonfiguration als eine Matrix aus Integers (den Sandhöhen) darstellen.

Auf diesem diskretiserten Modell betrachten wir zwei verschiedene Prozesse:
1.  Das Ablagern von neuem Sand, welches zu Rutschungen führen kann
2.  Das Transportieren von Sand durch Wind.

Diese Prozesse sind zunächst einmal unabhängig von einander implementierbar und werden im Folgenden getrennt erläutert.

### Sand Plotten

Bevor wir mit möglicher Dynamik beginnen, möchten wir Ihnen folgende Funktion zur Verfügung stellen, um eine gegebene Sand-Matrix zu plotten. Damit können Sie im Folgenden Teilschritte Ihres Codes testen.

In [ ]:
using PyPlot
using ColorTypes
pygui(false)

function show_sand(sand)
    # create a new figure
    fig = figure(figsize=(size(sand,1), size(sand,2)),dpi=5, frameon=false)
    # show the dunes
    duneimage = imshow(
        sand',
        cmap=ColorMap("Dunes", [RGB(0,0,0),RGB(0.7,0.6,0.3),RGB(1,1,1)]),
        interpolation="none",
        origin="lower"
    )
    # dont show axis
    gca().set_axis_off()
    # set margins to 0 (so that the image fills the whole frame)
    gcf().subplots_adjust(bottom=0, top=1, right=1, left=0)
end

### Sand Dynamik

Wir betrachten nun zuerst nur die Dynamik beim Ablagern eines Sandhäufchens auf eine bestehende Menge Sand. Dabei kann es zu verschiedene Szenarien kommen, die in folgender Skizze zusammengefasst werden:



In Bild a) erkennt man einen Querschnitt durch die zweidimensionale Sandkonfiguration links. Sand ist durch beige Kästchen dargestellt. Jedem Feld $i,j$ ist eine Höhe $h_{ij} \in \mathbb{N}$, also die Anzahl der darauf vorhandenen Sandhäufchen, zugeordnet (links dargestellt durch Zahlen in den Feldern).

Das Ablagern eines neuen Sandhäufchens auf ein bestimmtes Feld (Bild b) ) führt zum Erhöhen der Sandhöhe auf diesem Feld um $1$ (Bild c) ). Im Beispiel ist die Sandhöhe auf diesem Feld jetzt von 1 auf 2 gestiegen ist.

Wird ein weiteres Häufchen auf das gleiche Feld abgelagert (Bild d) ), so wächst die Höhe des Sandes auf diesem Feld zuerst weiter um $1$ (im Beispiel von 2 auf 3, siehe Bild e) ). Wird jedoch der Gradient zu einem der vier Nachbarfelder zu groß, kommt es zu einem Rutschprozess und das Sandhäufchen wird entfernt und stattdessen auf dem entsprechenden Nachbarfeld abgelagert (Bild f) ).

Beachten Sie, dass dieser Rutsch-Prozess eine Kaskade von Ereignissen im Schema d)-e)-f) in Gang setzen kann, die erst aufhört, wenn kein weiterer zu großer Gradient bei Rutschen entsteht.

Implementieren Sie im Folgenden die Funktion `drop_sand!`, die auf eine übergebene Sandkonfiguration (gegeben als Array `sand`) ein neues Häufchen an die Stelle $i,j$ einfügt. Es kann sinnvoll sein die Funktion `drop_sand!` rekursiv zu benutzen sobald ein Rutsch-Prozess entsteht, sprich das Ablagern durch Rutschungen auf einem Nachbarfeld führt zu einem erneuten Aufruf von `drop_sand!`.

In [ ]:
function drop_sand!(sand, i,j)
    
end

Testen Sie Ihre Funktion, indem Sie die nachfolgende Zelle ausführen. Diese modelliert die Situation in der Sie einen Hügel aus Sand aufschütten. Es wird zuerst eine leere Konfiguration ohne Sand initialisiert und nachfolgend 10.000 Sandhäufchen auf einen kleinen Bereich fallen gelassen. Vergleichen Sie den resultierenden Hügel mit Ihren Erwartungen und stellen Sie an diesem Beispiel sicher, dass Ihre Funktion `drop_sand!` wie gewollt funktioniert.

In [ ]:
# Initialize Sand
sand = zeros(Int64,100,100)

# drop 10000 sand
for s in 1:2000
    drop_sand!(sand, 50+rand(1:10), 80+rand(1:10))
end

# plot configuration
show_sand(sand)

### Wind Dynamik

Wir betrachten nun zusätzlich die Dynamik durch Wind. Wind kann Sandhäufchen aufheben, transportieren und ablegen. Dabei wollen wir uns explizit auf Wind in der x-Richtung beschränken, d.h. Sand kann über den Wind nur in der x-Richtung transportiert werden. Das Ablegen funktioniert jedoch wieder über die bereits diskutierte Funktion `drop_sand!` und kann somit Kaskaden in jede Richtung auslösen.

Unser Modell des Windes funktioniert in folgenden Schritten:
1.  Es wird ein zufälliges Feld mit Sand ausgewählt, das nicht im Windschatten liegt. Ein Feld liegt dann im Windschatten, wenn sein Nachbar in der Richtung aus der der Wind kommt, mehr Sand besitzt.
2.  Ein Sandhäufchen wird durch den Wind aufgenommen (dadurch befindet sich nun weniger Sand auf dem Feld)
3.  Führe nachfolgende Punkte solange aus, bis der Sand abgelegt wurde:
    1.  Das Sandhäufchen wird um eine Anzahl Punkte in x-Richtung bewegt (noch nicht abgelegt)
    2.  Es wird versucht das Häufchen abzulegen:
        - Ist das erreichte Feld leer, wird der Sand mit einer Wahrscheinlichkeit `p_NS` abgelegt
        - Ist das erreichte Feld mit Sand belegt, wird der Sand mit einer Wahrscheinlichkeit `p_S` abgelegt

Dieser Algorithmus soll im Folgenden in der Funktion `transport_sand!` implementiert werden. Der Funktion wird erneut das Array mit der aktuellen Sandkonfiguration, `sand`, übergeben, sowie die Wahrscheinlichkeiten `p_S` und `p_NS`. Ausserdem kann die Windstärke über die Transportreichweite `wind` spezifiziert werden.

*Hinweis 1*: Eine zufällige, ganzzahlige Zahl zwischen 1 und N können Sie mit dem Befehl `rand(1:N)` erzeugen.

*Hinweis 2*: Ob ein Sandhäufchen mit der Wahrscheinlichkeit `p` abgelegt wird können in Julia Sie mittels der Bedingung `rand() < p` prüfen. Dabei erzeugt `rand()` eine reelle Zufallszahl zwischen 0 und 1 und prüft ob sie kleiner als die Wahrscheinlichkeit `p` ist.

In [ ]:
function transport_sand!(sand, p_S, p_NS, wind)
    
end

### Animation des Sandes

Für die Animation des Sandes stellen wir Ihnen im Folgenden einen Satz von Animationsfunktionen zur Verfügung. Führen Sie die Zellen einfach aus, damit die Funktionen definiert werden.

In [ ]:
using PyPlot
using ColorTypes
using PyCall
pygui(true);

In [ ]:
function plot_sand(sand, vmax, dpi)
    # create a new figure
    fig = figure(figsize=(size(sand,1), size(sand,2)),dpi=dpi, frameon=false)
    # show the dunes
    sand_image = imshow(
        sand',
        vmax=vmax,
        cmap=ColorMap("Dunes", [RGB(0,0,0),RGB(0.7,0.6,0.3),RGB(1,1,1)]),
        interpolation="none",
        origin="lower"
    )
    # dont show axis
    gca().set_axis_off()
    # set margins to 0 (so that the image fills the whole frame)
    gcf().subplots_adjust(bottom=0, top=1, right=1, left=0)
    # return the duneimage and the figure
    return (fig, sand_image)
end;

In [ ]:
function update_sand_image(sand_image, sand)
    # update image
    sand_image.set_array(sand')
    # pause and display the image
    pause(0.001)
end;

### Dünen Test

Im Folgenden können Sie nun Ihren kompletten Code testen. Dazu wird erst eine initiale Sandkonfiguration erzeugt. Danach wird eine Updatefunktion definiert, die die Konfiguration zwischen den Frames der Animation aktualisiert. Schließlich folgt die eigentliche Animation.

In [ ]:
#############
# PARAMETER #
#############

# Die Systemgröße
Lx = 192
Ly = 108
# Anzahl an Sandhäufchen, die zuerst verteilt werden
N  = 350000

# Ablegewahrscheinlichkeiten für wind
p_S  = 0.35
p_NS = 0.05
# Transportlänge durch Wind
wind = 3
# Wie viele Transportvorgänge pro Frame gemacht werden
wind_updates = 10000




#############
# PROGRAMM  #
#############

# Initiale Sandkonfiguration ohne Sand
sand = zeros(Int64,Lx,Ly)

# Sand-Startbedingungen erzeugen
for s in 1:N
    drop_sand!(sand, rand(1:Lx), rand(1:Ly))
end

# Erstes Plotten der Sandkonfiguration
(fig,sand_image) = plot_sand(sand, 50, 5)

# Animieren des Windes
while length(get_fignums()) > 0
    # updates durchführen
    for s in 1:wind_updates
        transport_sand!(sand, p_S,p_NS, wind)
    end
    # darstellung aktualisieren
    update_sand_image(sand_image, sand)
end

# Bild nach fertiger Animation schliessen
close(fig)